# Elementos do dataset

Antes mesmo de falar do projeto, ao longo do desenvolvimento, vamos explicitando qual o objetivo de cada versão de trabalho do arquivo principal.

O objetivo desta AvoidMort_ds4 é criar um dataset de mortalidades gerais para servir de comparação para a mortalidade evitável.

O dataset final deverá ter as seguintes características:


* setting: Brasil [poderemos encontrar um sufixo a Brasil que permita a sua subclassificação e carga deste dataset junto a outros]
* dimension: UF
* subgroup: municípios
* estimate: mortalidade por CIDs evitáveis e município
* setting-average: mortalidade por CIDs evitáveis por Brasil

# Projeto de Análise de Desigualdade em Saúde no modelo HEAT/OMS

## Bloco 2025/1 (mortalidade evitável)

Análise do sistema de mortalidade do SUS com agregação por unidades federativas

### Instalação e importação de bibliotecas

In [2]:
# importar biblioteca básicas para manipulação de dados
import pandas as pd
import numpy as np
import os
import sidrapy
import requests # para carga de endereços web - api
import gdown
import openpyxl
from tabulate import tabulate

In [3]:
# Para organizar resultados, gerar prints em amarelo

def print_y(text):
  """Imprime o texto fornecido na cor amarela.

  Args:
    text: O texto a ser impresso.
  """
  print(f"\033[33m ==> {text}\033[0m")

# Exemplo de uso:
print_y("Este texto será impresso em amarelo!")
print("Este em normal!")

 ==> Este texto será impresso em amarelo!
Este em normal!


# Merge de dados de CID e População no arquivo principal

### Merge de dados de CID

In [5]:
ocde_evit_temp = pd.read_csv('files_in_geral/cid10_evit_ocde.csv')
ocde_evit_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   avoid_flag               346 non-null    object
 1   prevent_flag             189 non-null    object
 2   treat_flag               184 non-null    object
 3   Group                    346 non-null    object
 4   Causes of deaths         346 non-null    object
 5   Rationale for inclusion  346 non-null    object
 6   Range                    346 non-null    object
 7   cid_pai                  346 non-null    object
dtypes: object(8)
memory usage: 21.8+ KB


In [6]:
# Gerar nova tabela com valores únicos de CAUSABAS
dobr_evit_merged_temp = pd.read_parquet('files_in_geral/dobr_10_19a22_u75.parquet')
causas_unicas = dobr_evit_merged_temp['CAUSABAS'].unique()

# Converter para DataFrame
df_causas_unicas = pd.DataFrame(causas_unicas, columns=['CAUSABAS'])

# Exibir as primeiras linhas do DataFrame
df_causas_unicas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11895 entries, 0 to 11894
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CAUSABAS  11895 non-null  object
dtypes: object(1)
memory usage: 93.1+ KB


##########         Etapa diferenciada da versão          ##########

Utilizar 'Causes of deaths' de CID evitáveis e as demais como "Causa não evitável"

In [7]:
# Fazer merge da lista de causas_unicas com a lista de causas evitáveis

df_a = df_causas_unicas
column_a = 'CAUSABAS'
df_b = ocde_evit_temp
column_b = 'cid_pai'
column_return = 'Causes of deaths'



# Função para verificar a correspondência e retornar a classificação
def verificar_correspondencia(row):
    for index, row_b in df_b.iterrows():
        if row[column_a].startswith(row_b[column_b]):
            return row_b[column_return]
    return 'causa não evitável'

# Aplicando a função a cada linha do DataFrame A
df_a['causes_deaths'] = df_a.apply(verificar_correspondencia, axis=1)

print(tabulate(df_a.head()))

-  ----  -------------------
0  A049  Intestinal diseases
1  A09   Intestinal diseases
2  A162  Tuberculosis
3  A169  Tuberculosis
4  A199  Tuberculosis
-  ----  -------------------


In [8]:
# Merge dobr_merged_pop_clean and df_a by CAUSABAS
dobr_evit_merged_temp01 = pd.merge(dobr_evit_merged_temp, df_a, on='CAUSABAS', how='left')

# mostrar dados gerais da tabela após transformações
tab = dobr_evit_merged_temp01
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 4427475
+----+------------+-------------+--------+-----------+--------------+-------------+---------------------+
|    | CAUSABAS   | CODMUNRES   |   SEXO |   RACACOR |   OBITO_SOMA | ANO_OBITO   | causes_deaths       |
|----+------------+-------------+--------+-----------+--------------+-------------+---------------------|
|  0 | A049       | _355080     |      1 |         1 |            1 | _2019       | Intestinal diseases |
|  1 | A09        | _320520     |      1 |         1 |            1 | _2019       | Intestinal diseases |
|  2 | A09        | _330040     |      1 |         1 |            1 | _2019       | Intestinal diseases |
|  3 | A09        | _350970     |      1 |         4 |            1 | _2019       | Intestinal diseases |
|  4 | A09        | _355100     |      2 |         1 |            1 | _2019       | Intestinal diseases |
+----+------------+-------------+--------+-----------+--------------+-------------+---------------------+


In [9]:
# Mostrar resultados de classificacao
dobr_evit_merged_temp01.groupby('causes_deaths').count()

,CAUSABAS,CODMUNRES,SEXO,RACACOR,OBITO_SOMA,ANO_OBITO
causes_deaths,,,,,,
Abdominal hernia,5886,5886,5886,5886,5886,5886
Abscess of lung and mediastinum pyothorax,2512,2512,2512,2512,2512,2512
Accidental Injuries,101006,101006,101006,101006,101006,101006
Acute lower respiratory infections,3261,3261,3261,3261,3261,3261
Acute pancreatitis,11836,11836,11836,11836,11836,11836
...,...,...,...,...,...,...
Varicella,270,270,270,270,270,270
Venous thromboembolism,24533,24533,24533,24533,24533,24533
Viral Hepatitis,8688,8688,8688,8688,8688,8688


########## Etapa diferenciada da versão ##########

Não serão excluídos os registros com CID de doenças não evitáveis

In [ ]:
# # Manter na base só dados de CID evitáveis
# dobr_evit_merged_temp02 = dobr_evit_merged_temp01[dobr_evit_merged_temp01['classificacao']=='Avoidable mortality']

# # mostrar dados gerais da tabela após transformações
# tab = dobr_evit_merged_temp02
# print(f'Registro da tabela : {tab.shape[0]}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

#### Resultados do merge de cid_avoid e dobr

##########       Atenção      ##########

Atenção para etapa de salvar arquivo, para não sobrescrever outra versão

In [10]:
# Salvar arquivo final de DOBR

dobr_evit_merged_temp01.to_parquet('files_temp/ds4_dobr_evit_merged.parquet')

In [11]:
# Contador

shape_origin = dobr_evit_merged_temp.CAUSABAS.count()
shape_filt = dobr_evit_merged_temp01.CAUSABAS.count()
dif = shape_origin - shape_filt


print_y(f" O número de óbitos após a limpeza e que constituía a base de dados de DO era de {shape_origin} óbitos ")
print_y(f" O número de registros excluídos por apresentar CIDs NÃO EVITÁVEIS foi de {dif} óbitos ")
print_y(f" O número de óbitos por CID EVITÁVEIS é de {shape_filt} óbitos ")

 ==>  O número de óbitos após a limpeza e que constituía a base de dados de DO era de 4427475 óbitos 
 ==>  O número de registros excluídos por apresentar CIDs NÃO EVITÁVEIS foi de 0 óbitos 
 ==>  O número de óbitos por CID EVITÁVEIS é de 4427475 óbitos 


### Merge de dados de Municípios ao arquivo principal do SIM

Serão utilizados os arquivos:

* população: 'files_clean/censo_pop_clean.csv'

* sim: 'files_clean/dobr_clean.csv'

Como a população está clivada em Município, Idade (previamente, todos abaixo de 75 anos), Sexo e Raça, estes campos serão utilizados de maneira conjunta para realizar o merge.

In [4]:
# Exibir arquivo dobr_clean
dobr_merged_temp = pd.read_parquet('files_temp/ds4_dobr_evit_merged.parquet')

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp
print(f'Registro da tabela : {tab.shape[0]}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 4427475
+----+------------+-------------+--------+-----------+--------------+-------------+---------------------+
|    | CAUSABAS   | CODMUNRES   |   SEXO |   RACACOR |   OBITO_SOMA | ANO_OBITO   | causes_deaths       |
|----+------------+-------------+--------+-----------+--------------+-------------+---------------------|
|  0 | A049       | _355080     |      1 |         1 |            1 | _2019       | Intestinal diseases |
|  1 | A09        | _320520     |      1 |         1 |            1 | _2019       | Intestinal diseases |
|  2 | A09        | _330040     |      1 |         1 |            1 | _2019       | Intestinal diseases |
|  3 | A09        | _350970     |      1 |         4 |            1 | _2019       | Intestinal diseases |
|  4 | A09        | _355100     |      2 |         1 |            1 | _2019       | Intestinal diseases |
+----+------------+-------------+--------+-----------+--------------+-------------+---------------------+


In [5]:
# # Gerar um campo de ano para proxy de 2022 (para ser aplicados aos anos de 19 a 22)
dobr_merged_temp01 = dobr_merged_temp.copy()

dobr_merged_temp01['ANO_PROXY'] = dobr_merged_temp01['ANO_OBITO'].apply(lambda x: '_2010' if x == '_2010' else '_2022')

# # Criar um campo string que concatene ANO, CODMUNRES
# dobr_merged_temp01['POP_MERGE'] = dobr_merged_temp01['ANO_PROXY'].astype(str) + dobr_merged_temp['CODMUNRES']

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp01
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Soma de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql')) 

Registros da tabela : 4427475
Soma de óbitos : 4446922
+----+------------+-------------+--------+-----------+--------------+-------------+---------------------+-------------+
|    | CAUSABAS   | CODMUNRES   |   SEXO |   RACACOR |   OBITO_SOMA | ANO_OBITO   | causes_deaths       | ANO_PROXY   |
|----+------------+-------------+--------+-----------+--------------+-------------+---------------------+-------------|
|  0 | A049       | _355080     |      1 |         1 |            1 | _2019       | Intestinal diseases | _2022       |
|  1 | A09        | _320520     |      1 |         1 |            1 | _2019       | Intestinal diseases | _2022       |
|  2 | A09        | _330040     |      1 |         1 |            1 | _2019       | Intestinal diseases | _2022       |
|  3 | A09        | _350970     |      1 |         4 |            1 | _2019       | Intestinal diseases | _2022       |
|  4 | A09        | _355100     |      2 |         1 |            1 | _2019       | Intestinal diseases |

In [6]:
# # Verificar operação acima
print(f'Anos de ANO_OBITO {dobr_merged_temp01['ANO_OBITO'].unique()}')
print(f'Anos de ANO_PROXY {dobr_merged_temp01['ANO_PROXY'].unique()}')
# print(f'Anos de Censo {censo_4merge_temp['Ano'].unique()}')

Anos de ANO_OBITO ['_2019' '_2020' '_2021' '_2022' '_2010']
Anos de ANO_PROXY ['_2022' '_2010']


NameError: name 'censo_4merge_temp' is not defined

##########       Atenção      ##########

Iremos excluir o CAUSABAS, mas agrupar por município, ano e cause_deaths.
O contador de mortes portanto está atrelado ao cause_deaths.

In [7]:
# Agrupar por POP_MERGE e somar os registros, não contabilizando mais por CAUSABAS
col_group= [
            'causes_deaths', 
            'CODMUNRES',
            # 'SEXO',
            # 'RACACOR',
            'ANO_OBITO',
            'ANO_PROXY'
            ]

dobr_merged_temp02 = dobr_merged_temp01.groupby(col_group)['OBITO_SOMA'].sum().reset_index()



# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp02
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Soma de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql')) 

Registros da tabela : 473555
Soma de óbitos : 4446922
+----+------------------+-------------+-------------+-------------+--------------+
|    | causes_deaths    | CODMUNRES   | ANO_OBITO   | ANO_PROXY   |   OBITO_SOMA |
|----+------------------+-------------+-------------+-------------+--------------|
|  0 | Abdominal hernia | _110009     | _2019       | _2022       |            1 |
|  1 | Abdominal hernia | _110010     | _2022       | _2022       |            1 |
|  2 | Abdominal hernia | _110011     | _2020       | _2022       |            1 |
|  3 | Abdominal hernia | _110012     | _2022       | _2022       |            1 |
|  4 | Abdominal hernia | _110015     | _2010       | _2010       |            1 |
+----+------------------+-------------+-------------+-------------+--------------+


In [8]:
# Verificar colunas de população no arquivo censo_pop_clean
censo_4merge_temp = pd.read_parquet('files_in_geral/censo_mun.parquet')

# mostrar dados gerais da tabela após transformações
tab = censo_4merge_temp
print(f'Registro da tabela : {tab.shape[0]}')
print(f'População em 2022: {tab[tab.Ano == '_2022'].pop_total.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registro da tabela : 111400
População em 2022: 203069637
+----+-------+----------+----------------------------+--------+---------------+-------------+---------------+------+---------+----------+
|    | Ano   | mun_co   | mun_no                     | Sexo   | Cor ou raça   |   pop_total |   pop_under74 | uf   | uf_co   | uf_no    |
|----+-------+----------+----------------------------+--------+---------------+-------------+---------------+------+---------+----------|
|  0 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Branca        |        5311 |          5202 | RO   | _11     | Rondônia |
|  1 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Preta         |         693 |           673 | RO   | _11     | Rondônia |
|  2 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Amarela       |          96 |            94 | RO   | _11     | Rondônia |
|  3 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | Homens | Parda         |        6323 |          6206 | RO   

In [9]:
# Verificação dos nomes das colunas
censo_4merge_temp.columns

Index(['Ano', 'mun_co', 'mun_no', 'Sexo', 'Cor ou raça', 'pop_total',
       'pop_under74', 'uf', 'uf_co', 'uf_no'],
      dtype='object')

In [10]:
# Agrupar por POP_MERGE e somar os registros, não contabilizando mais por CAUSABAS
col_group= [
            'Ano',
            'mun_co',
            'mun_no',
            # 'Sexo',
            # 'Cor ou raça',
            # 'pop_total',
            # 'pop_under74',
            'uf',
            # 'uf_co',
            'uf_no'
            ]

censo_4merge_temp01 = censo_4merge_temp.groupby(col_group).agg({ 'pop_total':'sum',
                                                                'pop_under74':'sum',}).reset_index()



# mostrar dados gerais da tabela após transformações
tab = censo_4merge_temp01
print(f'Registros da tabela : {tab.shape[0]}')
print(f'População em 2022: {tab[tab.Ano == '_2022'].pop_total.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql')) 



Registros da tabela : 11140
População em 2022: 203069637
+----+-------+----------+----------------------------+------+----------+-------------+---------------+
|    | Ano   | mun_co   | mun_no                     | uf   | uf_no    |   pop_total |   pop_under74 |
|----+-------+----------+----------------------------+------+----------+-------------+---------------|
|  0 | _2010 | _110001  | Alta Floresta D'Oeste (RO) | RO   | Rondônia |       24392 |         23948 |
|  1 | _2010 | _110002  | Ariquemes (RO)             | RO   | Rondônia |       90347 |         89031 |
|  2 | _2010 | _110003  | Cabixi (RO)                | RO   | Rondônia |        6313 |          6180 |
|  3 | _2010 | _110004  | Cacoal (RO)                | RO   | Rondônia |       78574 |         76947 |
|  4 | _2010 | _110005  | Cerejeiras (RO)            | RO   | Rondônia |       17029 |         16646 |
+----+-------+----------+----------------------------+------+----------+-------------+---------------+


In [15]:
censo_4merge_temp01['pop_merge'] = censo_4merge_temp01['Ano']+censo_4merge_temp01['mun_co']
dobr_merged_temp02['pop_merge'] = dobr_merged_temp02['ANO_PROXY']+dobr_merged_temp02['CODMUNRES']


In [16]:

# Realizar o merge dos dataframes
dobr_merged_temp03 = pd.merge(dobr_merged_temp02, censo_4merge_temp01, 
                              on = 'pop_merge',
                            #   left_on=['ANO_PROXY','CODMUNRES'],
                            #   right_on=['Ano','mun_co'],
                              how='left')


In [17]:

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp03
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Soma de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql')) 

Registros da tabela : 473555
Soma de óbitos : 4446922
+----+------------------+-------------+-------------+-------------+--------------+--------------+-------+----------+--------------------------+------+----------+-------------+---------------+
|    | causes_deaths    | CODMUNRES   | ANO_OBITO   | ANO_PROXY   |   OBITO_SOMA | pop_merge    | Ano   | mun_co   | mun_no                   | uf   | uf_no    |   pop_total |   pop_under74 |
|----+------------------+-------------+-------------+-------------+--------------+--------------+-------+----------+--------------------------+------+----------+-------------+---------------|
|  0 | Abdominal hernia | _110009     | _2019       | _2022       |            1 | _2022_110009 | _2022 | _110009  | Espigão D'Oeste (RO)     | RO   | Rondônia |       29414 |         28491 |
|  1 | Abdominal hernia | _110010     | _2022       | _2022       |            1 | _2022_110010 | _2022 | _110010  | Guajará-Mirim (RO)       | RO   | Rondônia |       39387 |   

In [19]:
# Filtrar registros onde pop_under74 não é nulo
dobr_merged_temp04 = dobr_merged_temp03.dropna(subset=['pop_under74']).reset_index(drop=True)

# mostrar dados gerais da tabela após transformações
tab = dobr_merged_temp04
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Soma de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 472557
Soma de óbitos : 4440843
+----+------------------+-------------+-------------+-------------+--------------+--------------+-------+----------+--------------------------+------+----------+-------------+---------------+
|    | causes_deaths    | CODMUNRES   | ANO_OBITO   | ANO_PROXY   |   OBITO_SOMA | pop_merge    | Ano   | mun_co   | mun_no                   | uf   | uf_no    |   pop_total |   pop_under74 |
|----+------------------+-------------+-------------+-------------+--------------+--------------+-------+----------+--------------------------+------+----------+-------------+---------------|
|  0 | Abdominal hernia | _110009     | _2019       | _2022       |            1 | _2022_110009 | _2022 | _110009  | Espigão D'Oeste (RO)     | RO   | Rondônia |       29414 |         28491 |
|  1 | Abdominal hernia | _110010     | _2022       | _2022       |            1 | _2022_110010 | _2022 | _110010  | Guajará-Mirim (RO)       | RO   | Rondônia |       39387 |   

##### Arquivo tratado final

In [20]:
dobr_merged_temp04.to_parquet('files_temp/ds4_dobr_merged.parquet')

In [22]:
# Resultados

obito_origin = dobr_merged_temp.CAUSABAS.count()
obito_final = dobr_merged_temp04.OBITO_SOMA.sum()
dif = obito_origin - obito_final


print_y(f" O número de óbitos após merge com tabela de CIDs evitáveis era de {obito_origin} óbitos ")
print_y(f" Foram excluídos, por apresentar problemas para carga de informações de município, {dif} óbitos ")
print_y(f" O número de óbitos total após filtro de CID evitáveis e inclusão de população do censo é de {obito_final} óbitos ")


# # Filtrar registros onde pop_under74 é null
# dobr_merged_pop_nulos = dobr_merged_pop[dobr_merged_pop['pop_under74'].isnull()]

# # Exibir os registros filtrados
# dobr_merged_pop_nulos['obito_count'].sum()

 ==>  O número de óbitos após merge com tabela de CIDs evitáveis era de 4427475 óbitos 
 ==>  Foram excluídos, por apresentar problemas para carga de informações de município, -13368 óbitos 
 ==>  O número de óbitos total após filtro de CID evitáveis e inclusão de população do censo é de 4440843 óbitos 


# Etapa 3 - Aplicação de Modelo do HEAT ao Bloco

O modelo HEAT apresenta um framework bem rígido e objetivo para carga de dados.

O dataset precisa estar organizado com os labels e de acordo com a definição presente no template.

E depois precisa ser validado em planilha excel disponibilizada pela organização.

Ajustes: com a identificação da população por sexo e raça, encontramos 37 registros com população 0. Tivemos que excluir estes registros (incluindo os óbitos) no momento da preparação do dataset do heat em razão do modelo não aceitar estes valores.

## Dataset 4 - BR/CIDs/uf/Mun


O primeiro dataset que iremos construir tem as seguintes características:

* set = Brasil
* indicador = causes_deaths
* dimen = UF
* subgroup = município
* estimate = mortalidade por ano, causa (indicador) e subgroup
* set-avg = mortalidade por ano e causa (indicador)

Estas escolhas implicam em podermos disponibilizar ao usuário do HEAT a possibilidade de ver os municípios (dimensões) de maneira agregada por UF. 


In [3]:
# Define the URL and the destination path
url = "https://srhdpeuwpubsa.blob.core.windows.net/whdh/HIDR/heat_plus_template_validation.xlsm"
destination_path = "downloads_outros/heat_plus_template_validation.xlsm"

# Download the file
response = requests.get(url)
with open(destination_path, 'wb') as file:
    file.write(response.content)

# Note: Enabling macros programmatically is not recommended due to security risks.
# Macros should be enabled manually in Excel settings.

In [24]:
# Abrir arquivo final de DO após tratamento
heatds_temp = pd.read_parquet('files_temp/ds4_dobr_merged.parquet')

# mostrar dados gerais da tabela após transformações
tab = heatds_temp
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 472557
Total de óbitos : 4440843
+----+------------------+-------------+-------------+-------------+--------------+--------------+-------+----------+--------------------------+------+----------+-------------+---------------+
|    | causes_deaths    | CODMUNRES   | ANO_OBITO   | ANO_PROXY   |   OBITO_SOMA | pop_merge    | Ano   | mun_co   | mun_no                   | uf   | uf_no    |   pop_total |   pop_under74 |
|----+------------------+-------------+-------------+-------------+--------------+--------------+-------+----------+--------------------------+------+----------+-------------+---------------|
|  0 | Abdominal hernia | _110009     | _2019       | _2022       |            1 | _2022_110009 | _2022 | _110009  | Espigão D'Oeste (RO)     | RO   | Rondônia |       29414 |         28491 |
|  1 | Abdominal hernia | _110010     | _2022       | _2022       |            1 | _2022_110010 | _2022 | _110010  | Guajará-Mirim (RO)       | RO   | Rondônia |       39387 |  

In [38]:
# # Agregar os dados sem CID e gerar colunas de soma de obito_count e pop_under74
# heat_semCID = heat_.groupby(['UF', 'mun_nome', 'CODMUNRES', 'SEXO', 'RACACOR', 'ANO']).agg({
#     'obito_count': 'sum',
#     'pop_under74': 'sum'
# }).reset_index()

########## __Atenção__ ##########

Para melhor registro, utilizamos o label mort_ind_mun_causedeath para deixar claro que o indicador é relativo a município e causa de morte

In [27]:
# Gerar o indicador de mortalidade (por 100 mil habitantes) e exibir as primeiras linhas do DataFrame
heatds_temp01 = heatds_temp.copy()

heatds_temp01['mort_ind_mun_causedeath'] = (heatds_temp01['OBITO_SOMA']/heatds_temp01['pop_under74'])*100000

# mostrar dados gerais da tabela após transformações
tab = heatds_temp01
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.OBITO_SOMA.sum()}')
print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 472557
Total de óbitos : 4440843
+----+------------------+-------------+-------------+-------------+--------------+--------------+-------+----------+--------------------------+------+----------+-------------+---------------+---------------------------+
|    | causes_deaths    | CODMUNRES   | ANO_OBITO   | ANO_PROXY   |   OBITO_SOMA | pop_merge    | Ano   | mun_co   | mun_no                   | uf   | uf_no    |   pop_total |   pop_under74 |   mort_ind_mun_causedeath |
|----+------------------+-------------+-------------+-------------+--------------+--------------+-------+----------+--------------------------+------+----------+-------------+---------------+---------------------------|
|  0 | Abdominal hernia | _110009     | _2019       | _2022       |            1 | _2022_110009 | _2022 | _110009  | Espigão D'Oeste (RO)     | RO   | Rondônia |       29414 |         28491 |                  3.50988  |
|  1 | Abdominal hernia | _110010     | _2022       | _2022      

In [28]:
# Contar e gerar tabela de indicador de mortalidade infinitos [inf]
registros_inf = heatds_temp01[np.isinf(heatds_temp01['mort_ind_mun_causedeath'])]
inf_count = np.isinf(heatds_temp01['mort_ind_mun_causedeath']).sum()
nulos_mort_ind_mun = heatds_temp01['mort_ind_mun_causedeath'].isnull().sum()


# Exibir os registros filtrados
print(f"Número de valores infinitos em mort_ind_mun: {inf_count}")
print(f"Valores nulos em 'mort_ind_mun': {nulos_mort_ind_mun}")



# Exportar registros de indicador de mortalidade infinitos [inf]
# registros_inf.to_csv('exportar/relatorio_reg_mort_ind_inf.csv')

Número de valores infinitos em mort_ind_mun: 0
Valores nulos em 'mort_ind_mun': 0


In [ ]:
# # Limpar a base de registros de indicador de mortalidade infinitos [inf]
# heat_ds_treat2 = heat_ds_treat.copy()

# # Substituir valores infinitos por NaN
# heat_ds_treat2['mort_ind_mun'].replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
# # Rodar se for necessário excluir valores nulos
# heatds_temp02 = heatds_temp01.copy()

# # Excluir registros com NaN na coluna mort_ind_mun
# heatds_temp02.dropna(subset=['mort_ind_mun'], inplace=True)

# # Mudar o nome da coluna mort_ind para mort_ind_mun
# # heat_ds_treat2.rename(columns={'mort_ind':'mort_ind_mun'}, inplace=True)


# # Verificar as primeiras linhas do DataFrame atualizado
# heat_ds_treat1.head()

In [29]:
# Verificar se existe valor infinito ou negativo na coluna mort_ind_mun
heatds_temp01.describe()

,OBITO_SOMA,pop_total,pop_under74,mort_ind_mun_causedeath
count,472557.000000,4.725570e+05,4.725570e+05,472557.000000
mean,9.397476,8.621225e+04,8.285124e+04,26.701832
std,117.927717,4.057112e+05,3.874540e+05,49.169391
min,1.000000,8.050000e+02,7.820000e+02,0.009178
25%,1.000000,1.002300e+04,9.544000e+03,4.990393
50%,2.000000,2.069400e+04,1.975300e+04,12.059817
75%,5.000000,5.184500e+04,4.993700e+04,26.271166
max,35890.000000,1.145115e+07,1.089595e+07,870.195794


##### Agrupar por UF para gerar setting e setting-average

De acordo com o template do HEAT, o setting average: "must be consistent for the same combinantion of: setting, year, source and indicator".

Isto implica que os valores para cada setting (UF) não traz interferência nem da dimensão (município) e nem do subgrupo (sexo-raça-cor).

In [ ]:
# # Agrupar por 'UF', 'ANO', e calcular a média de 'mort_ind'
# dobr_grouped_uf = heat_ds_treat2.groupby(['UF',
#                                              'ANO_OBITO',
#                                             #  'SEXO',
#                                             #  'RACACOR'
#                                              ]).agg({'CONTADOR': 'sum',
#                                                     'pop_under74': 'sum',
#                                                     'mort_ind_mun': 'mean'
#                                                 }).reset_index()
# dobr_grouped_uf.rename(columns={'mort_ind_mun': 'mort_ind_uf',
#                                 'CONTADOR': 'obito_count_uf',
#                                 'pop_under74': 'pop_under74_uf',
#                                 }, inplace=True)


# # Exibir as primeiras linhas do DataFrame resultante
# dobr_grouped_uf.head()

In [ ]:
# dobr_grouped_uf.describe()

In [ ]:
# Salvar arquivo por UF para exportação
# dobr_grouped_uf.to_excel('exportar/dobr_grouped_uf_v2.xlsx', index=False)

In [ ]:
# heat_ds_treat2.columns

In [ ]:
# # Trazer dados por UF (setting e setting_average)

# # Realizar o merge entre heat_ e uf_sett
# heat_treat00 = pd.merge(heat_ds_treat2, dobr_grouped_uf, on=['UF',
#                                                                 'ANO_OBITO',
#                                                                 # 'SEXO',
#                                                                 # 'RACACOR'
#                                                                 ], how='left')

# # Exibir as primeiras linhas do dataframe resultante
# heat_treat00.head()

In [ ]:
# # Teste de aplicação de setting_average (médias por UF)
# filtered_data = heat_treat00[   (heat_treat00['ANO_OBITO'] == '_2022') &
#                                 (heat_treat00['UF'] == 'AC') 
#                                 # & (heat_treat00['SEXO'] == '_1') &
#                                 #(heat_treat00['RACACOR'] == '_4')
#                               ]
# filtered_data.head()

In [ ]:
# heat_treat00.columns

In [ ]:
# heat_treat00.pop_under74_uf.sum()

In [ ]:
# obito_num = heat_treat00.CONTADOR.sum()
# pop_under74 = heat_treat00.pop_under74.sum()

# print(f"\033[33m ==> Número de mortes: {obito_num} \033[0m")
# print(f"\033[33m ==> Número de população abaixo de 75 anos é de {pop_under74} \033[0m")  


In [ ]:
# heat_template = row
# heat_template

##### Criar colunas do dataset

##########       Etapa diferenciada da versão      ##########

Esta etapa precisa bastante atenção.

Se considerarmos causes_deaths como dimensão, ela precisa ser ignorada na criação so set-avg.

Pelo contrário, se formos tratar causes_deaths como indicador, ele precisa ser considerado na criação do set-avg.

Vamos realizar abaixo causes_deaths como indicador.

In [30]:
heatds_temp01.columns

Index(['causes_deaths', 'CODMUNRES', 'ANO_OBITO', 'ANO_PROXY', 'OBITO_SOMA',
       'pop_merge', 'Ano', 'mun_co', 'mun_no', 'uf', 'uf_no', 'pop_total',
       'pop_under74', 'mort_ind_mun_causedeath'],
      dtype='object')

In [31]:
# Na média ponderada de mortalidade evitável, pegamos os valores de mortalidade para cada município e dividimos pela população, para conseguir o valor para o Brasil e por ano

# Função para calcular a média ponderada
def weighted_avg(group):
    return np.average(group['mort_ind_mun_causedeath'], weights=group['pop_under74'])

# Agrupar os dados pelo ano e aplicar a função de média ponderada
weighted_avg_per_year = heatds_temp01.groupby(['ANO_OBITO','causes_deaths']).apply(weighted_avg).reset_index()

# Renomear as colunas para melhor entendimento
weighted_avg_per_year.columns = ['ANO_OBITO','causes_deaths','set_avg_BR']

# Exibir o resultado
print(weighted_avg_per_year)

    ANO_OBITO                              causes_deaths  set_avg_BR
0       _2010                           Abdominal hernia    0.903182
1       _2010  Abscess of lung and mediastinum pyothorax    0.673781
2       _2010                        Accidental Injuries   11.020761
3       _2010         Acute lower respiratory infections    0.579969
4       _2010                         Acute pancreatitis    1.917004
..        ...                                        ...         ...
393     _2022               Upper respiratory infections    0.653991
394     _2022                                  Varicella    0.409445
395     _2022                     Venous thromboembolism    4.069469
396     _2022                            Viral Hepatitis    1.232398
397     _2022                         causa não evitável  175.127151

[398 rows x 3 columns]


/tmp/ipykernel_9265/1189972450.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_avg_per_year = heatds_temp01.groupby(['ANO_OBITO','causes_deaths']).apply(weighted_avg).reset_index()


In [32]:
# Realizar o merge entre heat_ds_treat2 e grouped_data usando a coluna ANO_OBITO
heatds_temp02 = pd.merge(heatds_temp01, weighted_avg_per_year, on=['ANO_OBITO','causes_deaths'], how='left')

# mostrar dados gerais da tabela após transformações
tab = heatds_temp02
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.OBITO_SOMA.sum()}')
tab.head()
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))

Registros da tabela : 472557
Total de óbitos : 4440843


,causes_deaths,CODMUNRES,ANO_OBITO,ANO_PROXY,OBITO_SOMA,pop_merge,Ano,mun_co,mun_no,uf,uf_no,pop_total,pop_under74,mort_ind_mun_causedeath,set_avg_BR
0,Abdominal hernia,_110009,_2019,_2022,1,_2022_110009,_2022,_110009,Espigão D'Oeste (RO),RO,Rondônia,29414.0,28491.0,3.509880,1.136874
1,Abdominal hernia,_110010,_2022,_2022,1,_2022_110010,_2022,_110010,Guajará-Mirim (RO),RO,Rondônia,39387.0,38368.0,2.606339,1.243599
2,Abdominal hernia,_110011,_2020,_2022,1,_2022_110011,_2022,_110011,Jaru (RO),RO,Rondônia,50589.0,48848.0,2.047167,1.050270
3,Abdominal hernia,_110012,_2022,_2022,1,_2022_110012,_2022,_110012,Ji-Paraná (RO),RO,Rondônia,124333.0,120643.0,0.828892,1.243599
4,Abdominal hernia,_110015,_2010,_2010,1,_2010_110015,_2010,_110015,Ouro Preto do Oeste (RO),RO,Rondônia,37928.0,37114.0,2.694401,0.903182


##########       Etapa diferenciada da versão      ##########

Para esta versão
* set = Brasil
* indicador = causes_deaths
* dimen = UF
* subgroup = município
* estimate = mortalidade por ano, causa (indicador) e subgroup
* set-avg = mortalidade por ano e causa (indicador)

In [33]:
# Etapas para gerar indicator_abbr exclusivos

# Etapa 1 - Gerar uma tabela apenas com valores exclusivos de heatds_temp02.causes_deaths, em ordem crescente
unique_causes_deaths = heatds_temp02[['causes_deaths']].drop_duplicates().sort_values(by='causes_deaths').reset_index(drop=True)

# Etapa 2 - Criar coluna indicator_abbr com os primeiros 5 caracteres de causes_deaths
unique_causes_deaths['indicator_abbr'] = unique_causes_deaths['causes_deaths'].str[:5]

# Etapa 3 - Para aqueles indicator_abbr que tiverem o mesmo nome, gerar sufixo incremental a partir de 1
unique_causes_deaths['indicator_abbr'] = unique_causes_deaths.groupby('indicator_abbr').cumcount().astype(str).radd(unique_causes_deaths['indicator_abbr'])

# Etapa 3.1 - excluir valores que terminam com 0
unique_causes_deaths['indicator_abbr'] = unique_causes_deaths['indicator_abbr'].str.replace('0', '')

# Exibir a tabela resultante
unique_causes_deaths

,causes_deaths,indicator_abbr
0,Abdominal hernia,Abdom
1,Abscess of lung and mediastinum pyothorax,Absce
2,Accidental Injuries,Accid
3,Acute lower respiratory infections,Acute
4,Acute pancreatitis,Acute1
...,...,...
76,Varicella,Varic
77,Venous thromboembolism,Venou
78,Viral Hepatitis,Viral
79,Whooping cough,Whoop


In [35]:
# Fazer merge com informações de indicator_abbr exclusivos
heatds_temp03 = pd.merge(heatds_temp02,unique_causes_deaths,on='causes_deaths', how='left')

# mostrar dados gerais da tabela após transformações
tab = heatds_temp03
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.OBITO_SOMA.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))
tab.head()

Registros da tabela : 472557
Total de óbitos : 4440843


,causes_deaths,CODMUNRES,ANO_OBITO,ANO_PROXY,OBITO_SOMA,pop_merge,Ano,mun_co,mun_no,uf,uf_no,pop_total,pop_under74,mort_ind_mun_causedeath,set_avg_BR,indicator_abbr
0,Abdominal hernia,_110009,_2019,_2022,1,_2022_110009,_2022,_110009,Espigão D'Oeste (RO),RO,Rondônia,29414.0,28491.0,3.509880,1.136874,Abdom
1,Abdominal hernia,_110010,_2022,_2022,1,_2022_110010,_2022,_110010,Guajará-Mirim (RO),RO,Rondônia,39387.0,38368.0,2.606339,1.243599,Abdom
2,Abdominal hernia,_110011,_2020,_2022,1,_2022_110011,_2022,_110011,Jaru (RO),RO,Rondônia,50589.0,48848.0,2.047167,1.050270,Abdom
3,Abdominal hernia,_110012,_2022,_2022,1,_2022_110012,_2022,_110012,Ji-Paraná (RO),RO,Rondônia,124333.0,120643.0,0.828892,1.243599,Abdom
4,Abdominal hernia,_110015,_2010,_2010,1,_2010_110015,_2010,_110015,Ouro Preto do Oeste (RO),RO,Rondônia,37928.0,37114.0,2.694401,0.903182,Abdom


In [41]:
# Gerar colunas sem informação ou com informação padronizada

# heat_treat01 = heat_semCID_clean # caso não seja agrupado por UF anteriormente
heatds_temp04 = heatds_temp03.copy() # caso seja agrupado por UF anteriormente

heatds_temp04['setting'] = 'Brasil'
# heat_treat01['setting_average'] = set_avg_BR

# heatds_temp03['indicator_abbr'] = 'AVM'
# heatds_temp03['indicator_abbr'] = heatds_temp03['causes_deaths'].str.slice(0, 5)
# heatds_temp03['indicator_name'] = 'Avoidable mortality (deaths per 100 000 population)'
heatds_temp04['indicator_name'] = heatds_temp04['causes_deaths'] + '(deaths per 100 000 population)'
heatds_temp04['source'] = 'SIM/SUS & Censo/IBGE'
heatds_temp04['favourable_indicator'] = 0
heatds_temp04['indicator_scale'] = 100000
heatds_temp04['ordered_dimension'] = 0
heatds_temp04['subgroup_order'] = 0
heatds_temp04['reference_subgroup'] = 0
heatds_temp04['note'] = ''
heatds_temp04['se'] = ''
heatds_temp04['ci_lb'] = ''
heatds_temp04['ci_ub'] = ''
heatds_temp04['iso3'] = 'BRA'

# Renomear colunas de heat_treat01
heatds_temp04 = heatds_temp04.rename(columns={
    'ANO_OBITO':'date',
    'pop_under74':'population',
    'uf':'dimension',
    'mun_no':'subgroup',
    # 'mort_ind_mun':'estimate',
    'mort_ind_mun_causedeath':'estimate',
    'set_avg_BR':'setting_average',
    # 'mort_ind_uf':'setting_average'
})

# Ajustar coluna de ano
heatds_temp04['date'] = heatds_temp04['date'].str.replace('_','').astype(int)


# mostrar dados gerais da tabela após transformações
tab = heatds_temp04
print(f'Registros da tabela : {tab.shape[0]}')
print(f'Total de óbitos : {tab.OBITO_SOMA.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))
tab.head()


Registros da tabela : 472557
Total de óbitos : 4440843


,causes_deaths,CODMUNRES,date,ANO_PROXY,OBITO_SOMA,pop_merge,Ano,mun_co,subgroup,dimension,...,favourable_indicator,indicator_scale,ordered_dimension,subgroup_order,reference_subgroup,note,se,ci_lb,ci_ub,iso3
0,Abdominal hernia,_110009,2019,_2022,1,_2022_110009,_2022,_110009,Espigão D'Oeste (RO),RO,...,0,100000,0,0,0,,,,,BRA
1,Abdominal hernia,_110010,2022,_2022,1,_2022_110010,_2022,_110010,Guajará-Mirim (RO),RO,...,0,100000,0,0,0,,,,,BRA
2,Abdominal hernia,_110011,2020,_2022,1,_2022_110011,_2022,_110011,Jaru (RO),RO,...,0,100000,0,0,0,,,,,BRA
3,Abdominal hernia,_110012,2022,_2022,1,_2022_110012,_2022,_110012,Ji-Paraná (RO),RO,...,0,100000,0,0,0,,,,,BRA
4,Abdominal hernia,_110015,2010,_2010,1,_2010_110015,_2010,_110015,Ouro Preto do Oeste (RO),RO,...,0,100000,0,0,0,,,,,BRA


In [285]:
# # Trazer labels para sexo e raçacor
# heat_treat02 = heat_treat01.copy()

# heat_treat02['SEXO'] = heat_treat02['SEXO'].map({
#                                             '_1':'Homens',
#                                             '_2':'Mulheres'
#                                             })
# heat_treat02['RACACOR'] = heat_treat02['RACACOR'].map({
#                                             '_1':'Branca',
#                                             '_2':'Preta',
#                                             '_3':'Amarela',
#                                             '_4':'Parda',
#                                             '_5':'Indígena',
#                                             '_9':'Sem declaração'
#                                             })

# heat_treat02['subgroup'] = heat_treat02['SEXO'] + ' - raça/cor '  +  heat_treat02['RACACOR']


In [ ]:
# heat_treat02['subgroup'].unique()

In [ ]:
# heat_treat02.columns

In [42]:
# Define the order of columns
column_order = [
    'setting',
    'date',
    'source',
    'indicator_abbr',
    'indicator_name',
    'dimension',
    'subgroup',
    'estimate',
    'se',
    'ci_lb',
    'ci_ub',
    'population',
    'note',
    'setting_average',
    'iso3',
    'favourable_indicator',
    'indicator_scale',
    'ordered_dimension',
    'subgroup_order',
    'reference_subgroup'
    ]  


# Reorder the columns in the DataFrame
heatds_temp05 = heatds_temp04[column_order]


In [43]:
# Transformar estimate e setting_average em int
heatds_temp06 = heatds_temp05.copy()

heatds_temp06['estimate'] = heatds_temp06['estimate'].astype(int)
heatds_temp06['setting_average'] = heatds_temp06['setting_average'].astype(int)

# mostrar dados gerais da tabela após transformações
tab = heatds_temp06
print(f'Registros da tabela : {tab.shape[0]}')
# print(f'Total de óbitos : {tab.CONTADOR.sum()}')
# print(tabulate(tab.head(), headers='keys', tablefmt='psql'))
tab.head()

Registros da tabela : 472557


,setting,date,source,indicator_abbr,indicator_name,dimension,subgroup,estimate,se,ci_lb,ci_ub,population,note,setting_average,iso3,favourable_indicator,indicator_scale,ordered_dimension,subgroup_order,reference_subgroup
0,Brasil,2019,SIM/SUS & Censo/IBGE,Abdom,Abdominal hernia(deaths per 100 000 population),RO,Espigão D'Oeste (RO),3,,,,28491.0,,1,BRA,0,100000,0,0,0
1,Brasil,2022,SIM/SUS & Censo/IBGE,Abdom,Abdominal hernia(deaths per 100 000 population),RO,Guajará-Mirim (RO),2,,,,38368.0,,1,BRA,0,100000,0,0,0
2,Brasil,2020,SIM/SUS & Censo/IBGE,Abdom,Abdominal hernia(deaths per 100 000 population),RO,Jaru (RO),2,,,,48848.0,,1,BRA,0,100000,0,0,0
3,Brasil,2022,SIM/SUS & Censo/IBGE,Abdom,Abdominal hernia(deaths per 100 000 population),RO,Ji-Paraná (RO),0,,,,120643.0,,1,BRA,0,100000,0,0,0
4,Brasil,2010,SIM/SUS & Censo/IBGE,Abdom,Abdominal hernia(deaths per 100 000 population),RO,Ouro Preto do Oeste (RO),2,,,,37114.0,,0,BRA,0,100000,0,0,0


##########       Etapa diferenciada da versão      ##########

Atenção da hora de salvar o arquivo, com as informações corretas de versão

In [44]:
### Arquivo pronto para exportação
heatds_temp06.to_excel('files_out/ds4_BR_CID_mun.xlsx', index=False)

In [95]:
# Gerar arquivo dataset diretamente no arquivo de validação do HEAT

# Carregar o arquivo original
workbook = openpyxl.load_workbook('files_out/heat_plus_template_validation.xlsm', keep_vba=True)

# Apagar a aba 'template'
if 'template' in workbook.sheetnames:
    del workbook['template']

# Carregar o arquivo que será copiado como nova aba
workbook_to_copy = openpyxl.load_workbook('files_out/AvoidMort_v4_ds_4val.xlsx')
sheet_to_copy = workbook_to_copy.active

# Copiar a aba para o workbook original
new_sheet = workbook.create_sheet('template')

for row in sheet_to_copy.iter_rows(values_only=True):
    new_sheet.append(row)

# Salvar o arquivo com o novo nome
workbook.save('files_out/AvoidMort_v4_heat_plus_template_ds_4val.xlsm')

# Etapa 4 - Reunião de alinhamento - ajustes

# Etapa 5 - Preparação para publicação de resultados